In [1]:
# imports
import tkinter as tk
from tkinter import ttk
import colorsys
from PIL import ImageGrab, Image, ImageTk, ImageEnhance
import random
import string
from time import sleep
from numpy.random import choice
import numpy as np

from tqdm import tqdm

import json
import os

# import matplotlib
# matplotlib.use("TkAgg")
# from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from io import BytesIO
# import numpy as np

In [2]:
def norm(x):
    return [i/sum(x) for i in x]

In [3]:
# ----------------------------------------
# constants
MIN_WIDTH = 80
MAX_WIDTH = 200

MIN_HEIGHT = 30
MAX_HEIGHT = 100

LARGE_CELL_PROB = 0.6
LARGE_CELL_SIZES = [3,5]
MERGE_RATE_EDGE = 0.1
MERGE_RATE = 0.05

widget_names = ["button", "label", "entry"]
WIDGET_LIST = ["empty", "button", "label", "entry"]
WIDGET_PROB = norm([100,0.1,0.1,0.1])
WIDGET_EDGE_PROB = norm([100,10,10,5])
WIDGET_NEIGHBOUR_PROB = norm([100,10,10,5])

LARGE_WIDGET_LIST = ["table", "plot", "video"]
LARGE_WIDGET_PROB = norm([1,1,1])

BACKGROUND_PROB = 0.6

# WIDGET_LIST = ["empty", "label", "button"]
# WIDGET_PROB = [0.0, 0.5, 0.5]

In [4]:
def random_string(min_len, max_len):
    length = random.randint(min_len, max_len)
    letters = string.ascii_letters  # a-z + A-Z
    return ''.join(random.choice(letters) for _ in range(length))

In [5]:
def save_annotation_json(iter_idx, screen_width, screen_height, cell_list, color_dict,
                         screenshot_filename, output_dir="annotations"):

    os.makedirs(output_dir, exist_ok=True)

    json_data = {
        "image": screenshot_filename,
        "resolution": {
            "width": screen_width,
            "height": screen_height
        },
        "widgets": []
    }

    # Convert all widgets into proper annotation entries
    id_counter = {w:1 for w in widget_names}

    for c in cell_list:

        if c.widget == "table" and c.table is not None:
            json_data["widgets"].append({
                "id": "table",
                "type": "Table",
                "bbox": {
                    "x_min": int(c.lx),
                    "y_min": int(c.uy),
                    "x_max": int(c.rx),
                    "y_max": int(c.dy)
                },
                "structure": {
                    "rows": c.table["rows"],
                    "cols": c.table["cols"],
                    "headers": c.table["headers"],
                    "data": c.table["data"]
                }
            })
            continue

        if c.widget == "video":
            json_data["widgets"].append({
                "id": "video",
                "type": "Video",
                "bbox": {
                    "x_min": int(c.lx),
                    "y_min": int(c.uy),
                    "x_max": int(c.rx),
                    "y_max": int(c.dy)
                }
            })
            continue

        if c.widget == "plot":
            json_data["widgets"].append({
                "id": "plot",
                "type": "Plot",
                "bbox": {
                    "x_min": int(c.lx),
                    "y_min": int(c.uy),
                    "x_max": int(c.rx),
                    "y_max": int(c.dy)
                }
            })
            continue


        if c.widget in widget_names:
            widget_id = f"{c.widget}_{id_counter[c.widget]}"
            id_counter[c.widget] += 1

            # If you want to store actual widget text:
            # Here you only know the template names; the actual ttk widgets
            # have already been created in the GUI. But the label/button text
            # is deterministic, so we reconstruct it.

            json_data["widgets"].append({
                "id": widget_id,
                "type": c.widget.capitalize(),
                "text": c.text,
                "color": color_dict[c.widget],
                "bbox": {
                    "x_min": int(c.lx),
                    "y_min": int(c.uy),
                    "x_max": int(c.rx),
                    "y_max": int(c.dy)
                }
            })

    # Save JSON file
    json_path = os.path.join(output_dir, f"annotation_{iter_idx}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2)

In [6]:
def hsl_to_hex(h, s, l):
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    return "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))

GUI_BASE_HUES = [
    0.55,  # muted blue
    0.33,  # green
    0.08,  # orange
    0.75,  # purple
    0.0,   # red
    0.16,  # brown/tan
    0.6,   # teal
    0.9    # pink/magenta
]

def generate_color_palette(n, palette_size=5, repeat_chance=0.3, white_chance=0.2, grey_chance=0.15):
    """
    Generate a harmonious GUI-friendly color palette with a chance for white.
    
    Parameters:
        n (int): Number of colors to generate.
        palette_size (int): Number of base colors in the palette.
        repeat_chance (float): Probability of repeating a color.
        white_chance (float): Probability of a palette color being white.
    """
    palette = []
    for i in range(palette_size):
        if random.random() < white_chance:
            palette.append("#ffffff")  # Add white directly
        elif random.random() < grey_chance:
            palette.append("#b2b2b2")  # Add grey directly
        else:
            h = random.choice(GUI_BASE_HUES)
            s = random.uniform(0.2, 0.45)
            l = random.uniform(0.5, 0.85)
            palette.append(hsl_to_hex(h, s, l))
    
    # Now build the final list with some repeats
    colors = []
    for _ in range(n):
        if colors and random.random() < repeat_chance:
            colors.append(random.choice(colors))
        else:
            colors.append(random.choice(palette))
    
    return colors

In [ ]:
for iter in tqdm(range(500)):
    iter += 265

    n = 15
    color_list = generate_color_palette(n, palette_size=len(WIDGET_LIST), repeat_chance=0.2)
    color_dict = {i:j for i,j in zip(WIDGET_LIST, color_list)}
    # print(color_list)

    # ----------------------------------------
    # the widget grid
    # ----------------------------------------

    root = tk.Tk()
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    root.destroy()
    root.mainloop()

    class cell:

        def __init__(self, lx, rx, uy, dy, ):

            self.lx = lx
            self.rx = rx
            self.uy = uy
            self.dy = dy

            self.edge = (lx == 0) or (uy == 0) or\
                        (rx == screen_width) or (dy == screen_height)
            
            self.type = "regular"
            self.active = True
            self.merged = False
            self.widget = ""
            self.text = ""
            self.table = None
            self.neighbours = []

    # generating rows and columns

    # columns
    row_coords = [0]
    column_coords = [0]

    while 1:
        new_x = random.randint(MIN_WIDTH, MAX_WIDTH)

        if (row_coords[-1] + new_x) > screen_width:
            # row_coords.append( screen_width) # uncommented: can result in small cells
            row_coords[-1] = screen_width
            break
        else:
            row_coords.append( row_coords[-1] + new_x)

    while 1:
        new_y = random.randint(MIN_HEIGHT, MAX_HEIGHT)

        if (column_coords[-1] + new_y) > screen_height:
            # column_coords.append( screen_height) # uncommented: can result in small cells
            column_coords[-1] = screen_height
            break
        else:
            column_coords.append( column_coords[-1] + new_y)

    # print(row_coords)
    # print(column_coords)

    # creating cells within a grid

    grid = []
    cell_list = []

    for i,r in enumerate(row_coords[:-1]):
        grid.append([])
        for j,c in enumerate(column_coords[:-1]):

            new_cell = cell(r, row_coords[i+1],
                            c,column_coords[j+1])
            
            grid[-1].append(new_cell)
            cell_list.append(new_cell)


    # finding neighbours and merging

    max_row = len(grid) - 1
    max_col = len(grid[0]) - 1


    def add_neighbours(c):
        if row > 0:
            c.neighbours.append( grid[row-1][col])
        if row < max_row:
            c.neighbours.append( grid[row+1][col])
        if col > 0:
            c.neighbours.append( grid[row][col-1])
        if col < max_col:
            c.neighbours.append( grid[row][col+1])
        return

    def assign_random_table(c,
                            min_rows=3,
                            max_rows=10,
                            min_cols=2,
                            max_cols=6,
                            min_text_len=3,
                            max_text_len=12):

        rows = random.randint(min_rows, max_rows)
        cols = random.randint(min_cols, max_cols)

        headers = [random_string(3, 8) for _ in range(cols)]

        data = []
        for _ in range(rows):
            row = [random_string(min_text_len, max_text_len) for _ in range(cols)]
            data.append(row)

        c.table = {
            "rows": rows,
            "cols": cols,
            "headers": headers,
            "data": data
        }

        c.widget = choice( LARGE_WIDGET_LIST, 1, p=LARGE_WIDGET_PROB)
        c.text = ""  # not used for tables

    def create_large_cell(grid, cell_size=4):
        # print("grid length", len(grid))
        # print("grid[0] length", len(grid[0]))

        if ((len(grid)-cell_size) < 1) or ((len(grid[0])-cell_size) < 1):
            return

        # starting (upper left corner) cell
        s_row = random.randint(1,len(grid)-cell_size)
        s_col = random.randint(1,len(grid[0])-cell_size)
        # transforming into list pos
        s_row -= 1
        s_col -= 1

        s_cell = grid[s_row][s_col]
        s_cell.type = "large"
        s_cell.rx = grid[s_row+cell_size][s_col].rx
        s_cell.dy = grid[s_row][s_col+cell_size].dy

        # print("s_row s_col", s_row, s_col)
        # print("lx", s_cell.lx)
        # print("uy", s_cell.uy)
        # print("rx", s_cell.rx)
        # print("dy", s_cell.dy)

        # setting all merge cells as not active
        for r in range(s_row,s_row+cell_size+1):
            for c in range(s_col,s_col+cell_size+1):
                grid[r][c].merged = True
                grid[r][c].active = False

        s_cell.active = True

        # assigning a table to the widget
        assign_random_table( s_cell)

    def merge_cells(c):
        for n in c.neighbours:
            if (not c.merged) and (not n.merged):
                merge_chance = random.random()

                if merge_chance < MERGE_RATE:
                    n.active = False
                    n.merged = True
                    c.merged = True
                    c.lx = min( c.lx, n.lx)
                    c.uy = min( c.uy, n.uy)
                    c.rx = max( c.rx, n.rx)
                    c.dy = max( c.dy, n.dy)
        return

    def assign_widget(c):
        if len([n.widget for n in c.neighbours if n.widget not in ["","empty"]]):
            widget = choice( WIDGET_LIST, 1, p=WIDGET_NEIGHBOUR_PROB)
        elif c.edge:
            widget = choice( WIDGET_LIST, 1, p=WIDGET_EDGE_PROB)
        else:
            widget = choice( WIDGET_LIST, 1, p=WIDGET_PROB)
            
        c.widget = widget[0]

        if c.widget == "button":
            c.text = random_string(5, 15)
        elif c.widget == "label":
            c.text = random_string(5, 15)
        return

    # gui creation

    def create_table_widget(parent, c):
        table = c.table
        if table is None:
            return

        tree = ttk.Treeview(
            parent,
            columns=[f"c{i}" for i in range(table["cols"])],
            show="headings"
        )

        for i, header in enumerate(table["headers"]):
            tree.heading(f"c{i}", text=header)
            tree.column(f"c{i}", anchor="center", width=80)

        for row in table["data"]:
            tree.insert("", "end", values=row)

        tree.place(
            x=c.lx,
            y=c.uy,
            width=c.rx - c.lx,
            height=c.dy - c.uy
        )

    if random.random() < LARGE_CELL_PROB:
        # creating a large cell
        create_large_cell(grid, cell_size=random.randint( LARGE_CELL_SIZES[0], LARGE_CELL_SIZES[1]))

    for row,_ in enumerate(grid):
        for col,_ in enumerate(grid[row]):
            c = grid[row][col]

            if (c.active) and (c.type == "regular"):
                add_neighbours(c)
                merge_cells(c)
                assign_widget(c)

    cell_list = []

    for row,_ in enumerate(grid):
        for col,_ in enumerate(grid[row]):
            c = grid[row][col]

            if c.active:
                cell_list.append(c)


    def image_enhancement(img):
        # Random rotation
        if random.random() < 0.6:
            img = img.rotate(random.choice([0, 90, 180, 270]), expand=True)

        # Color saturation change
        if random.random() < 0.8:
            img = ImageEnhance.Color(img).enhance(random.uniform(0.2, 1.8))

        # Contrast adjustment
        if random.random() < 0.8:
            img = ImageEnhance.Contrast(img).enhance(random.uniform(0.7, 1.4))

        # Brightness tweak
        if random.random() < 0.6:
            img = ImageEnhance.Brightness(img).enhance(random.uniform(0.7, 1.3))

        return img
    
    def generate_random_plot(width_px, height_px, dpi=100):
        """
        Generates a random matplotlib figure and returns a PIL image.
        
        Args:
            width_px, height_px: target pixel size of the plot
            dpi: dots per inch for the figure
        Returns:
            PIL.Image.Image object
        """
        # Convert pixels to inches for figsize
        width_in = width_px / dpi
        height_in = height_px / dpi
        
        # Create figure
        fig, ax = plt.subplots(figsize=(width_in, height_in), dpi=dpi, facecolor='white')
        
        # Random number of series
        n_series = random.randint(1, 3)
        
        # Random plot type
        plot_type = random.choices(
            ["line", "scatter", "bar"], weights=[0.5, 0.3, 0.2], k=1
        )[0]
        
        for i in range(n_series):
            n_points = random.randint(20, 50)
            x = np.linspace(0, 10, n_points)
            # Random y data
            y = np.random.uniform(-1, 1, size=n_points)
            color = random.choice(['blue', 'red', 'green', 'purple', 'orange'])
            
            if plot_type == "line":
                ax.plot(x, y, color=color, linewidth=random.uniform(1,3), alpha=random.uniform(0.7,1))
            elif plot_type == "scatter":
                ax.scatter(x, y, color=color, s=random.randint(20,80), alpha=random.uniform(0.5,1))
            elif plot_type == "bar":
                width = 0.15
                ax.bar(x + i*width, y, width=width, color=color, alpha=random.uniform(0.5,1))
        
        # # Randomize axes visibility
        # if random.random() < 0.7:
        #     ax.set_xticks([])
        # if random.random() < 0.7:
        #     ax.set_yticks([])
        # if random.random() < 0.5:
        #     ax.set_frame_on(False)

        # Randomize axes/frame visibility
        hide_x, hide_y, hide_frame = random.random() < 0.3, random.random() < 0.3, random.random() < 0.3

        # Ensure at least one is visible
        if hide_x and hide_y and hide_frame:
            keep = random.choice(["x","y","f"])
            hide_x, hide_y, hide_frame = keep != "x", keep != "y", keep != "f"

        # Apply
        if hide_x: ax.set_xticks([])
        if hide_y: ax.set_yticks([])
        ax.set_frame_on(not hide_frame)
        
        # Save to PIL image using BytesIO
        from io import BytesIO
        buf = BytesIO()
        fig.savefig(buf, format='png', pad_inches=0) # , bbox_inches='tight'
        buf.seek(0)
        img = Image.open(buf)
        img.load()  # force reading data
        buf.close()
        plt.close(fig)
        
        return img

    class MainApplication(tk.Frame):
        def __init__(self, parent, *args, **kwargs):
            tk.Frame.__init__(self, parent, background=color_list[0], *args, **kwargs)

            if random.random() < BACKGROUND_PROB:
                img = Image.open(f"backgrounds/{random.randint(0,23)}.jpg")

                # distorting image
                img = image_enhancement(img)

                img = img.resize((int(screen_width * random.uniform(1, 3)),
                                  int(screen_height * random.uniform(1, 3))), Image.LANCZOS)
                self.bg_img = ImageTk.PhotoImage(img)

                self.bg_label = tk.Label(self, image=self.bg_img)
                self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)

            self.parent = parent

            for c in cell_list:

                # large cell
                if c.widget == "table":
                    create_table_widget(self, c)

                if c.widget == "video":
                    # canvas approach to preserve transparency
                    canvas = tk.Canvas(self, highlightthickness=0)
                    canvas.place(x=c.lx, y=c.uy, width=c.rx - c.lx, height=c.dy - c.uy)

                    bg_width = c.rx - c.lx
                    bg_height = c.dy - c.uy

                    # --- Background image ---
                    video_img = Image.open(f"backgrounds/{random.randint(0,23)}.jpg")
                    video_img = image_enhancement(video_img)
                    video_img = video_img.resize((bg_width, bg_height), Image.LANCZOS)

                    video_tk = ImageTk.PhotoImage(video_img)
                    canvas.video_img = video_tk  # keep reference
                    canvas.create_image(0, 0, image=video_tk, anchor="nw")

                    # --- YouTube button overlay (transparent PNG) ---
                    width = int(bg_width / 5)
                    height = int((443 / 640) * width)

                    yt_img = Image.open("yt_button.png").convert("RGBA")
                    yt_img = yt_img.resize((width, height), Image.LANCZOS)

                    yt_tk = ImageTk.PhotoImage(yt_img)
                    canvas.yt_img = yt_tk  # keep reference

                    yt_item = canvas.create_image(
                        2 * width,
                        int((bg_height - height) / 2),
                        image=yt_tk,
                        anchor="nw"
                    )

                if c.widget == "plot":
                    width = c.rx - c.lx
                    height = c.dy - c.uy
                    img = generate_random_plot(width, height)
                    
                    # Convert to Tkinter image
                    img_tk = ImageTk.PhotoImage(img)
                    plot_label = tk.Label(self, image=img_tk, bd=0)
                    plot_label.image = img_tk
                    plot_label.place(x=c.lx, y=c.uy, width=width, height=height)


                if c.widget == "button":
                    ttk.Button( self, text=c.text).place(x=c.lx,y=c.uy, width=c.rx-c.lx, height=c.dy-c.uy)
                if c.widget == "label":
                    ttk.Label( self, text=c.text).place(x=c.lx, y=c.uy, width=c.rx-c.lx, height=c.dy-c.uy)
                if c.widget == "entry":
                    ttk.Entry( self).place(x=c.lx, y=c.uy, width=c.rx-c.lx, height=c.dy-c.uy)

    def take_screenshot_and_quit(root):
        root.update()
        screenshot = ImageGrab.grab()
        screenshot.save(f"screenshots/screenshot_{iter}.png")
        screenshot.close()
        root.destroy()

    if __name__ == "__main__":
        root = tk.Tk()

        # random style
        style = ttk.Style(root)
        themes = style.theme_names() # available themes
        style.theme_use(random.choice(themes))

        # Configure styles for each widget type
        style.configure("TButton", background=color_dict["button"], foreground="black")
        style.configure("TLabel", background=color_dict["label"], foreground="black")
        style.configure("TEntry", fieldbackground=color_dict["entry"], foreground="black")

        root.geometry(f"{screen_width}x{screen_height}")
        root.attributes("-fullscreen", True)
        root.attributes("-topmost", True)
        MainApplication(root).pack(side="top", fill="both", expand=True)

        root.after(500, lambda: take_screenshot_and_quit(root))

        root.mainloop()

    screenshot_filename = f"screenshots/screenshot_{iter}.png"

    save_annotation_json(
        iter_idx=iter,
        screen_width=screen_width,
        screen_height=screen_height,
        cell_list=cell_list,
        color_dict=color_dict,
        screenshot_filename=screenshot_filename
    )

 36%|███▌      | 180/500 [05:21<07:07,  1.34s/it]